In this notebook, we shall reproduce Livewire algorithm and compute the shortest path between two clicks.

# Input
read a DICOM slice image from file, convert it to a numpy array (http://www.osirix-viewer.com/datasets/)

In [2]:
import skimage

In [3]:
import dicom

In [4]:
df = dicom.read_file('IM-0001-0011.dcm')

In [5]:
image = skimage.img_as_uint(df.pixel_array)

In [6]:
print image, image[int(image.shape[0]*0.5)]

[[1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]
 ..., 
 [1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]
 [1619 1619 1619 ..., 1619 1619 1619]] [1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619 1619
 1619 1619 1619 1619 1650 1641 1615 1633 1619 1624 1618 1616 1629 1620 1620
 1618 1630 1612 1636 1628 1648 1629 1642 1641 1636 1636 1603 1662 1596 1745
 1811 1860 1996 2214 2339 2216 1915 1680 1664 1770 1828 1784 1739 1755 1778
 1746 1684 1660 1671 1663 1633 1640 1722 1821 1860 1828 1796 1794 1796 1762
 1715 1688 1691 1686 1663 1643 1643 1652 1647 1628 1619 1640 1671 1680 1653
 1616 1603 1625 1654 1678 1704 1747 1792 1798 1751 1677 1637 1640 1656 1656
 164

# Compute gradient image 
(http://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.sobel)

In [7]:
from skimage import filters

In [8]:
edges = filters.sobel(image)*1000

Display image

In [9]:
from skimage import io
import pylab
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


Convert gradient image into a graph

In [10]:
from math import fabs

G = {}
edges.shape
rows, cols = edges.shape
for col in range(cols):
    for row in range(rows):
        
        neighbors = []
        if row > 0:
            neighbors.append( (row-1, col) )
        
        if row < rows-1:
            neighbors.append( (row+1, col) )
            
        if col > 0:
            neighbors.append( (row, col-1) )
        
        if col < cols-1:
            neighbors.append( (row, col+1) )
        
        dist = {}
        for n in neighbors:
            n[0], n[1]
            dist[n] = fabs(edges[row][col] - edges[n[0], n[1]])
            
        G[(row,col)] = dist

In [17]:
print G[(180,180)], G[(190, 190)], shortestPath2(G, (180,180), (190, 190))

{(180, 181): 0.15345142558883618, (181, 180): 0.2896672286546787, (179, 180): 0.3045418918532188, (180, 179): 0.10523589975985659} {(189, 190): 0.15640940221356003, (191, 190): 0.29328522576869753, (190, 191): 0.035938875808507464, (190, 189): 0.026994842509516392} [(180, 180), (181, 180), (181, 181), (181, 182), (182, 182), (182, 183), (183, 183), (184, 183), (185, 183), (185, 184), (186, 184), (186, 185), (186, 186), (186, 187), (187, 187), (188, 187), (188, 188), (189, 188), (190, 188), (190, 189), (190, 190)]


Re-use Dijkstra algorithm (http://code.activestate.com/recipes/119466-dijkstras-algorithm-for-shortest-paths/)

In [11]:
from dijkstra import shortestPath, shortestPath2

In [12]:
shortestPath2(G, (257, 315), (232, 310))

[(257, 315),
 (256, 315),
 (255, 315),
 (254, 315),
 (254, 314),
 (253, 314),
 (253, 313),
 (252, 313),
 (251, 313),
 (251, 312),
 (250, 312),
 (249, 312),
 (249, 311),
 (248, 311),
 (248, 310),
 (247, 310),
 (246, 310),
 (245, 310),
 (244, 310),
 (243, 310),
 (242, 310),
 (241, 310),
 (240, 310),
 (239, 310),
 (238, 310),
 (237, 310),
 (236, 310),
 (235, 310),
 (234, 310),
 (233, 310),
 (232, 310)]

In [13]:
start_point = None
print start_point

def button_pressed(event):
    global start_point
    
    print event.xdata, event.ydata
    if start_point is None:
        start_point = (int(event.xdata), int(event.ydata))
        print 'first click'
        
    else:
        print 'second click'
        path = shortestPath2(G, start_point, (int(event.xdata), int(event.ydata)))
        print path
        start_point = None
        plt.plot(np.array(path)[:,0], np.array(path)[:,1])

import numpy as np
io.imshow(edges, cmap=plt.cm.gray )
plt.connect('button_release_event', button_pressed)
io.show()

None


/Library/Python/2.7/site-packages/skimage/io/_plugins/matplotlib_plugin.py:75: UserWarning: Float image out of standard range; displaying image with stretched contrast.
  warnings.warn("Float image out of standard range; displaying image "


266.433333333 110.166666667
first click
347.766666667 144.833333333
second click
[(266, 110), (265, 110), (265, 109), (265, 108), (265, 107), (265, 106), (266, 106), (267, 106), (268, 106), (269, 106), (270, 106), (271, 106), (272, 106), (272, 107), (273, 107), (273, 108), (273, 109), (274, 109), (275, 109), (276, 109), (277, 109), (278, 109), (279, 109), (280, 109), (281, 109), (282, 109), (283, 109), (284, 109), (284, 110), (285, 110), (286, 110), (287, 110), (288, 110), (289, 110), (290, 110), (291, 110), (291, 111), (291, 112), (291, 113), (291, 114), (291, 115), (291, 116), (291, 117), (291, 118), (291, 119), (291, 120), (291, 121), (291, 122), (291, 123), (291, 124), (291, 125), (291, 126), (291, 127), (291, 128), (291, 129), (291, 130), (291, 131), (291, 132), (291, 133), (291, 134), (291, 135), (291, 136), (291, 137), (291, 138), (291, 139), (291, 140), (291, 141), (291, 142), (291, 143), (291, 144), (292, 144), (293, 144), (294, 144), (295, 144), (296, 144), (297, 144), (298, 